## Movie Box Office Collection Data ETL Task

### Importing Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from datetime import datetime,timedelta
import math

### Movie List

In [3]:
import json
import os
abc=''
with open('config.py') as file:
    for i in file:
        abc+=i
file.close()
movielist=json.loads(abc)    

### Function for Creating connection with database and creating table in it

In [4]:
def create():
    conn = psycopg2.connect(database = "postgres", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")
    print ("Opened database successfully")
    cur = conn.cursor()

    cur.execute('''CREATE TABLE IF NOT EXISTS Boxoffice
          (Movie_Name Text     NOT NULL,
          Days_From_Release    TEXT,
          Date           TEXT     NOT NULL,
          Box_Office_Collection     FLOAT   );''')
    print ("Table created successfully")

    conn.commit()
    conn.close()

### Function for insering Data into database table 

In [5]:
def insert(p):
    conn = psycopg2.connect(database = "postgres", user = "postgres", password = "postgres", host = "127.0.0.1", port = "5432")
    #print("Opened database successfully")
    cur = conn.cursor()
    
    
    querry="INSERT INTO Boxoffice (Movie_Name,Days_From_Release,Date,Box_Office_Collection) \
          VALUES (%s,%s,%s,%s)"
    
    record=(i,Day[p],a_date[p],a_collection[p])

    cur.execute(querry,record);
    conn.commit()
    conn.close()

### Logic of crawling the data, Transforming the Data, loading the data into datbase

In [6]:
head='https://boxofficecollection.in/'
tail='-box-office-collection-day-wise'


for i,r in movielist.items():
    
    #Beautiful soup is used to extract the data from webpage
    webpage=requests.get(head+i+tail ).text
    soup=BeautifulSoup(webpage,'lxml')
    soup1=str(soup).split()
    
    #fetching table id of table
    tablename='tablepress-'
    for q in soup1:
        if 'tablepress-id-' in q:
            for s in q:
                if s.isnumeric():
                    tablename+=s
            break
    
    #fetching a particular table of box office collection
    table1 = soup.find('table', id=tablename)
    
    #fetching rows from table 
    row=[]
    for j in table1.find_all('tr')[1:]:
        row_data = j.find_all('td')
        
        #fetching Day and collection and appending in row list
        for k in row_data:
            row.append(k.text)
            
    #converting a release date in required format
    Release_date = r
    date=datetime.strptime(Release_date,'%Y-%m-%d').date()
                       
    
    #Segregetting Days and collection in different list
    Day=[]
    collection=[]
    for l in range (len(row)):
        if l % 2 == 0:
            Day.append(row[l])
        else:
            collection.append(row[l])
                                  
    #Auto increment of date accoring to Days 
    a_date=[]
    for m in (Day):
        if m.count('Day')>1:
            range1=m.split()
            D=[]
            for n in range(int(range1[-1])-int(range1[1])+1):
                date+=timedelta(days=1)
                D.append(str(date))
            a_date.append(D[0]+' to '+D[-1])
        else:
            if m=='Day 1':
                a_date.append(str(date))
            else:
                date+=timedelta(days=1)
                a_date.append(str(date))
                                  
    #Conversion of collection into required format(float)
    a_collection=[]
    for o in collection:
        x=o.split()
        y=x[0].split('₹')
        z=float(y[1])*10000000
        w=math.ceil(z)
        a_collection.append(w)
    
    #inserting a data into SQL table
    for p in range(len(Day)):
        insert(p)